In [ ]:
import sys
print(sys.executable)
import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator,FormatStrFormatter
import matplotlib.ticker as mtick
import seaborn as sns

from lifelines.datasets import load_waltons
from lifelines.statistics import multivariate_logrank_test
from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times

from sklearn.manifold import TSNE
import sklearn
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from lifelines import CoxPHFitter

In [ ]:
import time
import warnings

import numpy as np


from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

import scipy.stats as stats    #stats.ttest_ind(a=group1, b=group2, equal_var=True)

from itertools import combinations
from scipy import stats
from scipy.stats import mannwhitneyu

from scipy.stats import levene

### Select the cancer type form TCGA

####  Select the cancer type form TCGA using the condition that Pearson correlation coefficients (r) between the DeSide-predicted tumor purity and CPE values are greater than 0.6 and the RMSEs are less than 0.15.


In [ ]:
beter_pre = "HNSC,ACC,LUAD,CESC,UCEC,LIHC,KICH,PRAD,READ,GBM,KIRC,LGG,THCA,LUSC,BRCA,BLCA,PAAD,COAD,KIRP"
beter_pre_list = beter_pre.split(',')   


### Read cell proportions predicted by kassandra

In [ ]:

kassandra_path1 = r'D:\EMT\EMT\plot_231111\other_data\Kassandra_self_pred_cell_prop.csv'
kassandra_1 = pd.read_csv(kassandra_path1,index_col='sample_id')
kassandra_1 = kassandra_1[['cancer_type']]
kassandra_path = r'D:\EMT\EMT\plot_231111\other_data\all_result_TCGA_Kassandra.csv'
kassandra_data = pd.read_csv(kassandra_path,index_col=0)
kassandra_data = pd.merge(kassandra_1,kassandra_data,left_index=True,right_index=True)
print(kassandra_data.shape)
#kassandra_data.to_csv(r'D:\EMT\EMT\plot_231111\other_data\all_Kassandra_self_pred_cell_prop.csv')

kassandra_data.index = kassandra_data.index.str.rsplit('-',1).str[0]    # change index, for example: change "TCGA-C4-A0F1-01A" to "TCGA-C4-A0F1"
# kassandra_data  = kassandra_data.loc[:,['cancer_type', 'B Cells', 'CD4 T','CD8 T', 'Cancer Cells', 'Endothelial Cells', 'Fibroblasts','Macrophages', 'NK', 'Neutrophils'
# ]]

ori_kassandra_datasets ={}
for i in beter_pre_list:
    cf_new = kassandra_data.loc[kassandra_data['cancer_type']==i,:]
    cf_new = cf_new.drop('cancer_type',axis=1,inplace=False)
    print(i,'----------------')
    print('The number of duplicate samples: ',cf_new.index.duplicated().sum())
    print('Duplicate samples id:',cf_new.index[cf_new.index.duplicated(keep='first') ])
    cf_new = cf_new[~cf_new.index.duplicated(keep='first')]    #
    print('The number of samples after deleting the duplicated id: ',len(cf_new))
    ori_kassandra_datasets[i] = cf_new


### Read cell proportions predicted by deside

In [ ]:

my_datasets ={}
t_sne_datasets = {}
ori_datasets ={}
new_deside_data_path = r'D:\EMT\EMT\data\xiongxin_raw_data_0914\2023_\all_predicted_cell_fraction_by_DeSide_20231110_02_D1+D2.csv'
new_deside_data = pd.read_csv(new_deside_data_path,index_col='sample_id')
new_deside_data.index = new_deside_data.index.str.rsplit('-',1).str[0]    # change index, for example: change "TCGA-C4-A0F1-01A" to "TCGA-C4-A0F1"    
new_deside_data  = new_deside_data.loc[:,['cancer_type','Plasma B cells','Non-plasma B cells','CD4 T','CD8 T effector','CD8 T (GZMK high)','Double-neg-like T','Cancer Cells','DC','Endothelial Cells','CAFs','Myofibroblasts','Macrophages','Mast Cells','NK','Neutrophils','Monocytes'
]]
new_deside_data['B Cells'] = new_deside_data['Plasma B cells']+new_deside_data['Non-plasma B cells']
new_deside_data['CD8 T'] =new_deside_data['CD8 T effector']+new_deside_data['CD8 T (GZMK high)']
new_deside_data['Fibroblasts'] =new_deside_data['CAFs']+new_deside_data['Myofibroblasts']
new_deside_data['T Cells'] =new_deside_data['CD8 T']+new_deside_data['CD4 T']+ new_deside_data['Double-neg-like T']   #1207
new_deside_data['Lymphocytes'] =  new_deside_data['B Cells']+new_deside_data['T Cells']+new_deside_data['NK']
new_deside_data['Monocytic_cells']= new_deside_data['Monocytes']+new_deside_data['Macrophages']+new_deside_data['DC']   #https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0059243&type=printable
new_deside_data['Stromal_cells'] = new_deside_data['Fibroblasts']+new_deside_data['Endothelial Cells']
for i in beter_pre_list:
    cf_new = new_deside_data.loc[new_deside_data['cancer_type']==i,:]
    cf_new = cf_new.drop('cancer_type',axis=1,inplace=False)
    print(i,'----------------')
    print('The number of duplicate samples: ',cf_new.index.duplicated().sum())
    print('Duplicate samples id:',cf_new.index[cf_new.index.duplicated(keep='first') ])
    cf_new = cf_new[~cf_new.index.duplicated(keep='first')]    
    print('The number of samples after deleting the duplicated id: ',len(cf_new))
    ori_datasets[i] = cf_new
    cf_ = MinMaxScaler().fit_transform(cf_new)
    cf_ = pd.DataFrame(cf_,index=cf_new.index,columns= cf_new.columns)
    my_datasets[i] = cf_
    


### Dealing with input data for survival analysis

In [ ]:
#Dealing with input data for survival analysis, including 22 types of cancer.
# https://www.jianshu.com/p/f18f846ca602
#  For deceased samples, the "min(T,C)" column follows the "days_to_death".
# For surviving samples, the "min(T,C)" column follows the "days to last follow-up".

clinical_path = r'D:\EMT\EMT\data\xiongxin_raw_data_0914\2022\clinical_data\original_download\22_cancer_type\clinical.cases_selection.2023-03-23\clinical.csv'

clinical_df = pd.read_csv(clinical_path,index_col='case_submitter_id')
clinical_df = clinical_df[['days_to_death','vital_status','days_to_last_follow_up']]
clinical_df.replace("'--", np.NaN, inplace=True)
clinical_df['days_to_death'] = clinical_df['days_to_death'].astype('float64') 
clinical_df['days_to_last_follow_up'] = clinical_df['days_to_last_follow_up'].astype('float64') 

clinical_df = clinical_df[~clinical_df.index.duplicated(keep='first')]       # Delete the duplicated ID

print('The number of duplicate samples: ',clinical_df.index.duplicated().sum())



clinical_df.loc[clinical_df['vital_status']=='Alive','min(T,C)']=clinical_df['days_to_last_follow_up']  #
clinical_df.loc[clinical_df['vital_status']=='Dead','min(T,C)']=clinical_df['days_to_death']   #
clinical_df['vital_status'] = clinical_df['vital_status'].replace('Alive', 0)    # The statu of Alive is 0
clinical_df['vital_status'] = clinical_df['vital_status'].replace('Dead', 1)     # The statu of Dead is 1
print("The number of instances where the 'vital_status' column is marked as 'Not Reported' ：",clinical_df['vital_status'].isin(['Not Reported']).sum())
clinical_df =clinical_df[~clinical_df['vital_status'].isin(['Not Reported'])]   #Delete samples where the "vital_status" column is marked as "Not Reported".
clinical_df.dropna(axis=0,subset=["vital_status"],inplace=True)     #Delete samples where the "vital_status" column is empty.
clinical_df.dropna(axis=0,subset=["min(T,C)"],inplace=True)       ##Delete samples where the "min(T,C)" column is empty.             

clinical_df.loc[clinical_df.index,:]  
clinical_df = clinical_df.loc[clinical_df['min(T,C)']>=0,:]     
clinical_df = clinical_df.loc[clinical_df['min(T,C)']>0,:]    #Keep samples where the "min(T,C)" column is greater than 0.
#clinical_df['vital_status'].value_counts()   
#clinical_df.info()    
clinical_df                       

In [ ]:
def delet_miss_in_clinical(sub_cf,clinical_df=clinical_df):
    '''Delete cases from the subcluster that lack clinical information (such as LGG tumors) and return the clinical information of the corresponding cases. print.clinical_df
    '''
    judge_F = sub_cf.index.isin(clinical_df.index)

    if np.sum(judge_F==0)>0:   #0: False ,1: Ture

        missing_patient = sub_cf[~judge_F].index.tolist()
        sub_cf = sub_cf[judge_F]     
        print(f'The following patients were missing in original clinical data:{missing_patient}')
        print(f'Now,There are {len(sub_cf)} sample ')

    sub_clinical = clinical_df.loc[sub_cf.index,:]   
    if sub_clinical['vital_status'].isnull().any():
        number = sub_clinical['vital_status'].isnull().sum()
        sub_clinical.dropna(axis=0,subset=["vital_status"],inplace=True)     # axis=0， Delete in the row direction. 
        print(f'There were {number} na in vital_status colunm and were deleted.')
        print(f'Now,There are {len(sub_clinical)} id in cluster ')
    if sub_clinical['min(T,C)'].isnull().any():
        number = sub_clinical['min(T,C)'].isnull().sum()
        sub_clinical.dropna(axis=0,subset=["min(T,C)"],inplace=True)
        print(f'There were {number} na in min(T,C) colunm and were deleted.')
        print(f'Now,There are {len(sub_clinical)} id in cluster')
    if sub_clinical['vital_status'].isin(['Not Reported']).any():
        number = sub_clinical['vital_status'].isin(['Not Reported']).sum()
        sub_clinical =sub_clinical[~sub_clinical['vital_status'].isin(['Not Reported'])] 
        print(f'There were {number} str of Not Reported in vital_status colunm and were deleted.')
        print(f'Now,There are {len(sub_clinical)} id in cluster')

    return sub_clinical

def make_path(path):
    if not os.path.exists(path):
        os.makedirs(path)






In [ ]:
from matplotlib.pylab import mpl
# import scienceplots    # Whether to import depends on the version.
def set_fig_style(font_family=None, font_size=None):
    # set_fig_style(font_family='Arial', font_size=8)
    fig, ax = plt.subplots()
    
    try:
        sns.set_style("white")
        plt.style.use(['science','no-latex'])
    except:
        print('No science style')      
        sns.set(palette='muted', font_scale=1.5)
    sns.set_style("white")    
    mpl.rcParams['figure.dpi'] = 300
    mpl.rcParams['figure.facecolor'] = 'white'
    mpl.rcParams['pdf.fonttype'] = 42
    mpl.rcParams['ps.fonttype'] = 42
    plt.rcParams['svg.fonttype'] = 'none'
    if font_family:
        mpl.rcParams['font.family'] = font_family
    if font_size:
        mpl.rcParams['font.size'] = font_size  
    # print('figure.dpi will be set to', mpl.rcParams['figure.dpi'])
    plt.close('all')


### Patient stratified by two cell types

#### Plot all combination of two cell types

In [ ]:
%%time

clinical_data_all = {}    
median_in_dataset = {}
survival_function_in_dataset = {}

sing_p_in_datasets ={}
out_p_in_datasets = {}
overall_p_in_datasets ={}
median_s_t  =pd.DataFrame()
count_double_p=0
count_except=0
significant_logrank_cox_p = pd.DataFrame()
df_hr = pd.DataFrame()
for dataset_name in beter_pre_list:
#     save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\0622_4group'
#     save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\0908_4group_use_new_deside_results'
    save_path = r'.//results//4groups//1225_16CellType_add_median_survival'
    make_path(save_path)
    print(dataset_name,'********************************************')
    clinical_data_in_celltyppe ={}
    median_in_celltyppe  ={}
    survival_function_in_celltyppe = {}

    out_p_in_celltyppe = {}
    overall_p_value ={}   
    target_datasets=ori_datasets
    
    X = target_datasets[dataset_name]
#     X = X.rename(columns={'Endothelial Cells':'Endo','Fibroblasts':'Fb','Macrophages':'Mac'})   # alter name of some cell types 
#     X = X.rename(columns={'Endothelial Cells':'Endo','Fibroblasts':'Fb','Macrophages':'mø','CD4 T':'C4','CD8 T':'C8','B Cells':'B'})
    cancer_meidan = X.median()
    cancer_meidan_path = os.path.join(save_path,'result_data','cellfrac',f'{dataset_name}')
    make_path(cancer_meidan_path)
    cancer_meidan_path = os.path.join(cancer_meidan_path,'ct_median.xlsx')
    cancer_meidan.to_excel(cancer_meidan_path)
    
    com2=combinations(X.columns, 2)
    
    p_df = pd.DataFrame()
    median_celltype = pd.DataFrame()

    
    count_index = 0

    for ct_list in com2:
        print(ct_list)



        ct_i_list= {}

        list_vital_status=[]
        list_min_T_C =[]
        list_group =[]

        out_p_list ={}
        out_cox_p ={}
        out_HR = {}
        
        sub_clinical_data={}
        sub_survival_function = {}

        for ct_i in ct_list:






            media_ct_i = np.percentile(X.loc[:,ct_i], 50)
            ct_i_list[ct_i]=media_ct_i
        media1_name = list(ct_i_list.keys())[0]
        media2_name = list(ct_i_list.keys())[1]
        media1_value = ct_i_list[list(ct_i_list.keys())[0]]
        media2_value = ct_i_list[list(ct_i_list.keys())[1]]
        median_celltype.loc[dataset_name,media1_name] = media1_value
        median_celltype.loc[dataset_name,media2_name] = media2_value

        hh = X.loc[(X[media1_name]>ct_i_list[media1_name])&(X[media2_name]>ct_i_list[media2_name]),:]
        hl = X.loc[(X[media1_name]>ct_i_list[media1_name])&(X[media2_name]<=ct_i_list[media2_name]),:]
        lh = X.loc[(X[media1_name]<=ct_i_list[media1_name])&(X[media2_name]>ct_i_list[media2_name]),:]
        ll = X.loc[(X[media1_name]<=ct_i_list[media1_name])&(X[media2_name]<=ct_i_list[media2_name]),:]
        print(ct_list,'~~~~~~~~~~~~~~~~~~~~~~')
        print('Data number before filtering:',len(hh)+len(hl)+len(lh)+len(ll))
        print('hh')
        hh = delet_miss_in_clinical(hh)
        print('hl')
        hl = delet_miss_in_clinical(hl)
        print('lh')
        lh = delet_miss_in_clinical(lh)
        print('ll')
        ll = delet_miss_in_clinical(ll)
        print('Data number after filtering:',len(hh)+len(hl)+len(lh)+len(ll))

        zip_cluster = {'hh':hh,'hl':hl,'lh':lh,'ll':ll}


        list_vital_status = list(hh['vital_status'])+ list(hl['vital_status'])+list(lh['vital_status'])+list(ll['vital_status']) #used for caculating the general p_value
        list_min_T_C = list(hh['min(T,C)'])+ list(hl['min(T,C)'])+list(lh['min(T,C)'])+list(ll['min(T,C)'])          #used for caculating the general p_value
        list_group = ['hh']*len(hh['min(T,C)']) + ['hl']*len(hl['min(T,C)']) + ['lh']*len(lh['min(T,C)']) + ['ll']*len(ll['min(T,C)'])   #used for caculating the general p_value
        list_group_n = [0]*len(hh['min(T,C)']) + [1]*len(hl['min(T,C)']) + [2]*len(lh['min(T,C)']) + [3]*len(ll['min(T,C)'])
        list_vital_status_array = np.array(list_vital_status)
        list_min_T_C_array = np.array(list_min_T_C)
        list_group_array = np.array(list_group)

        list_cluster = ['hh','hl','lh','ll']    






        count_com5=0
        HR_com5 ={}

        com5=combinations(list_cluster, 2)
        for key_a,key_b in com5:   

            T1 = list_min_T_C_array[list_group_array==key_a]
            T2 = list_min_T_C_array[list_group_array==key_b]
            E1 = list_vital_status_array[list_group_array==key_a]
            E2 = list_vital_status_array[list_group_array==key_b]



            ######

            if (T1.shape[0]!=0)&(T2.shape[0]!=0):
                kmf = KaplanMeierFitter()
                kmf.fit(zip_cluster[key_a]['min(T,C)'], zip_cluster[key_a]['vital_status'])  #str(len(sub_clinical['min(T,C)'])) #data_race['min(T,C)'][[0,1,2,4]]                       
                median_a = np.round((kmf.median_survival_time_)/365,2)     

                kmf = KaplanMeierFitter()
                kmf.fit(zip_cluster[key_b]['min(T,C)'], zip_cluster[key_b]['vital_status'])  #str(len(sub_clinical['min(T,C)'])) #data_race['min(T,C)'][[0,1,2,4]]                       
                median_b = np.round((kmf.median_survival_time_)/365,2)     


                results = logrank_test(T1, T2, event_observed_A=E1, event_observed_B=E2)

                p_df.loc[str(count_index),'CancerType'] = dataset_name
                p_df.loc[str(count_index),'CellType'] = str(ct_list)
                p_df.loc[str(count_index),key_a+'_'+key_b] = f"{results.p_value:.1E}"                            #np.round(results.p_value,5)
                #!!!!!!!!!!!!!!!!!!!!!!!
                #calc hr



                d_a = zip_cluster[key_a]
                d_b = zip_cluster[key_b]
                d_a['group']=2  #The experimental group is labeled as "2" and serves as the numerator of the HR score.
                d_b['group']=1
                d_ab = pd.concat([d_a,d_b],axis=0,join='inner')
                d_ab = d_ab[['min(T,C)','vital_status','group']]
                try:
                    cph = CoxPHFitter()
                    cph.fit(d_ab, duration_col='min(T,C)', event_col ='vital_status')
                except:
                    count_except+=1
                    cph = CoxPHFitter(penalizer=0.1)
                    cph.fit(d_ab, duration_col='min(T,C)', event_col ='vital_status')                

                summary_  = cph.summary
                cox_p_lable = summary_['p'][0]  

                if cph.hazard_ratios_[0]<1:   
                    hr_larger_than_1 = 1/cph.hazard_ratios_[0]
    
                else:
                    hr_larger_than_1 = cph.hazard_ratios_[0]
       



                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                if (results.p_value<=0.05)&(cox_p_lable<=0.05):

                    count_double_p +=1
                    assemble = (key_a,key_b)
                    out_p_list[assemble]=np.round(results.p_value,2)
                    out_cox_p[assemble]=np.round(cox_p_lable,2)
                    out_HR[assemble]= np.round(hr_larger_than_1,1)
   
                    significant_logrank_cox_p.loc[count_double_p,'Cancer type']= dataset_name
                    significant_logrank_cox_p.loc[count_double_p,'Cell type']=ct_list[0]+'_'+ct_list[1]

                    significant_logrank_cox_p.loc[count_double_p,'subgroup']=key_a+'_'+key_b
                    significant_logrank_cox_p.loc[count_double_p,'subgroup sample number']=str(len(d_a))+'_'+str(len(d_b))
                    significant_logrank_cox_p.loc[count_double_p,'subgroup median survival']=str(median_a)+'_'+str(median_b)
                    significant_logrank_cox_p.loc[count_double_p,'logrank p']=results.p_value
                    significant_logrank_cox_p.loc[count_double_p,'HR']=hr_larger_than_1
                    significant_logrank_cox_p.loc[count_double_p,'cox_p']=cox_p_lable
   
            else:
                hr_larger_than_1 = float('-inf')
            HR_com5[key_a+'_'+key_b] = hr_larger_than_1
            df_hr.loc[dataset_name+'_'+ct_list[0]+'_'+ct_list[1],key_a+'_'+key_b]=hr_larger_than_1
            df_hr.loc[dataset_name+'_'+ct_list[0]+'_'+ct_list[1],'Cell type']=ct_list[0]+'_'+ct_list[1]
        hr_order1=sorted(HR_com5.items(),key=lambda x:x[1],reverse=False)

        hr_order = [i[0] for i in hr_order1]
        dict_hr =dict( zip(hr_order,range(1,7)))   # The maximum HR value is marked as 6, and the minimum HR value is marked as 1.
        dict_hr_ = dict( zip(range(1,7),hr_order))
        for i_ in range(1,7):
            df_hr.loc[dataset_name+'_'+ct_list[0]+'_'+ct_list[1],str(i_)] = dict_hr_[i_]


        for index_sig in significant_logrank_cox_p.loc[(significant_logrank_cox_p['Cancer type']==dataset_name)&(significant_logrank_cox_p['Cell type']==ct_list[0]+'_'+ct_list[1]),:].index:
            sig_ct = significant_logrank_cox_p.loc[index_sig,'subgroup']
            significant_logrank_cox_p.loc[index_sig,'ranking_HR']=dict_hr[sig_ct]


        set_fig_style(font_family='Arial', font_size=8)
        plt.figure(figsize=(8,8))
        for clu_name,clu_data in zip_cluster.items():

            if (clu_data['min(T,C)'].shape[0]!=0)&(clu_data['vital_status'].shape[0]!=0):
   
                legend_label=''
                if clu_name =='hh':
                    legend_label = 'h / h'
                if clu_name =='hl':
                    legend_label = 'h / l'
                if clu_name =='lh':
                    legend_label = 'l / h'
                if clu_name =='ll':
                    legend_label = 'l / l'
                kmf = KaplanMeierFitter()
                kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=f"{legend_label} (n={len(clu_data['min(T,C)'])})")  #str(len(sub_clinical['min(T,C)'])) #data_race['min(T,C)'][[0,1,2,4]]   
                kmf.plot(ci_show=False,show_censors=True)  #_false_interval
                median_ = kmf.median_survival_time_    
                median_s_t.loc[dataset_name,clu_name] = median_  ######

                df_survival_function = kmf.survival_function_
                
                sub_survival_function[clu_name] = df_survival_function
                plt.title(f'{dataset_name} ({media1_name}/{media2_name})')
                plt.ylabel('Survival probability')
                plt.xlabel('Overall survival(Days)')
                statistic_result = multivariate_logrank_test(list_min_T_C, list_group, list_vital_status)     
                p_value_= np.round(statistic_result.p_value,5)                 # general log rank p values

                p_df.loc[str(count_index),'general_p_value'] = p_value_


                plt.text(
                    0.01,
                    0.31,
                    (f"overall_log_p:{p_value_}"),
                    transform=plt.gca().transAxes,
                    horizontalalignment="left",
                    fontsize=12
                )

                plt.text(
                    0.01,
                    0.21,
                    (f"log_p:{out_p_list}"),
                    transform=plt.gca().transAxes,
                    horizontalalignment="left",
                    fontsize=12
                )

                plt.text(
                        0.01,
                        0.11,
                        (f"HR:{out_HR}"),
                        transform=plt.gca().transAxes,
                        horizontalalignment="left",
                        fontsize=12
                    ) 
                plt.text(
                        0.01,
                        0.01,
                        (f"cox_p:{out_cox_p}"),
                        transform=plt.gca().transAxes,
                        horizontalalignment="left",
                        fontsize=12
                    ) 



            sub_clinical_data[clu_name] = clu_data  
        count_index+=1      
        clinical_data_in_celltyppe[str(ct_list)] = sub_clinical_data
      
        survival_function_in_celltyppe[str(ct_list)] = sub_survival_function


        survival_plot_path = os.path.join(save_path,'plot_survival_curve',f'{dataset_name}')   
        make_path(survival_plot_path)
        survival_path = os.path.join(survival_plot_path,f'{str(ct_list)}.jpg')
        plt.savefig(survival_path)  


    
    p_df_path = os.path.join(survival_plot_path,dataset_name+'_p_value.xlsx')
    p_df.to_excel(p_df_path)

    median_s_t_path = os.path.join(survival_plot_path,'median_survival_time.xlsx')
    median_s_t.to_excel(median_s_t_path)

    median_ct_path = os.path.join(survival_plot_path,'median_celltype_frac.xlsx')
    median_celltype.to_excel(median_ct_path)
    

    survival_function_in_dataset[dataset_name] = survival_function_in_celltyppe
    clinical_data_all[dataset_name]= clinical_data_in_celltyppe  

 

      
significant_logrank_cox_p.loc[count_double_p,'count_except']=count_except
significant_logrank_cox_p_path =os.path.join(save_path,'significant_logrank_cox_p.xlsx')
significant_logrank_cox_p.to_excel(significant_logrank_cox_p_path)

df_hr_path = os.path.join(save_path,'df_hr.xlsx')
df_hr.to_excel(df_hr_path)
           


#### The cell composition used in the paper.

In [ ]:

# Fig.4b
target_keys_use=[
 ('LUSC', ('CAFs', 'Neut'),'lh','hh','low_high','high_high'),
 ('LUSC', ('CAFs','CD8 T effector'), 'hh','lh','high_high','low_high'),
 ('BLCA',('Non-plasma B', 'Neut'),'hl','hh','high_low','high_high'),
 ('LUAD',('MyoFb', 'Neut'),'hl','ll','high_low','low_low')
#  ('LUSC', "('Endo', 'Fb')",'ll','lh','low_low','low_high')
 ]

#suplemental imformation




#SIb
target_keys_use=[
 ('KIRP', ('Plasma B', 'CD8 T effector'),'hl','hh','high_low','high_high'),
 ('READ',('CD8 T effector','MyoFb'),'hh','lh','high_high','low_high'), 
('KIRP',('CD4 T','MyoFb'),'hh', 'lh', 'high_high', 'low_high'),    #'hh','hl','high_high','high_low'
#     ('KIRP',('Endo','MyoFb'),'hh', 'lh', 'high_high', 'low_high')   #'hh','hl','high_high','high_low'
 ]  


In [ ]:


mpl.rcParams['font.family'] = 'Arial'
sub_num = 0    #sub_num = 0  3  6   #改target_keys_use


#fig,ax = plt.subplots(3,3,figsize=(7,7.5),sharex=True, sharey=True)   #,sharey= True
fig,ax = plt.subplots(1,4,figsize=(7,1.7),sharex=True, sharey=True)   #,sharey= True
plt.subplots_adjust(hspace=0.2,wspace=0.05)  # wspace=0.25,

for tk in target_keys_use:
#     save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\0622_4group'
#     save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\0908_4group_use_new_deside_results'
    save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\2023\survival_4group\wenzhan'
    make_path(save_path)
    dataset_name = tk[0]
    print(dataset_name,'********************************************')

    target_datasets=ori_datasets  
    
    X = target_datasets[dataset_name]
    X = X.rename(columns={'Endothelial Cells':'Endo','Fibroblasts':'Fb','Macrophages':'mø','CD8 T':'C8','B Cells':'B',
                         'Non-plasma B cells':'Non-plasma B','Myofibroblasts':'MyoFb','Neutrophils':'Neut','Plasma B cells':'Plasma B'})   
    count_index = 0

    ct_list=tk[1]
    ct_i_list= {}
    list_vital_status=[]
    list_min_T_C =[]
    list_group =[]
    for ct_i in ct_list:
        
        media_ct_i = np.percentile(X.loc[:,ct_i], 50)
        ct_i_list[ct_i]=media_ct_i
    media1_name = list(ct_i_list.keys())[0]
    media2_name = list(ct_i_list.keys())[1]
    print(media1_name,media2_name)
    media1_value = ct_i_list[list(ct_i_list.keys())[0]]
    media2_value = ct_i_list[list(ct_i_list.keys())[1]]

    hh = X.loc[(X[media1_name]>ct_i_list[media1_name])&(X[media2_name]>ct_i_list[media2_name]),:]
    hl = X.loc[(X[media1_name]>ct_i_list[media1_name])&(X[media2_name]<=ct_i_list[media2_name]),:]
    lh = X.loc[(X[media1_name]<=ct_i_list[media1_name])&(X[media2_name]>ct_i_list[media2_name]),:]
    ll = X.loc[(X[media1_name]<=ct_i_list[media1_name])&(X[media2_name]<=ct_i_list[media2_name]),:]
    print(ct_list,'~~~~~~~~~~~~~~~~~~~~~~')
    print('Data number before filtering:',len(hh)+len(hl)+len(lh)+len(ll))
    print('hh')
    hh = delet_miss_in_clinical(hh)
    print('hl')
    hl = delet_miss_in_clinical(hl)
    print('lh')
    lh = delet_miss_in_clinical(lh)
    print('ll')
    ll = delet_miss_in_clinical(ll)
    print('Data number after filtering:',len(hh)+len(hl)+len(lh)+len(ll))
    zip_cluster = {'hh':hh,'hl':hl,'lh':lh,'ll':ll}


    list_vital_status = list(hh['vital_status'])+ list(hl['vital_status'])+list(lh['vital_status'])+list(ll['vital_status']) #used for caculating the general p_value
    list_min_T_C = list(hh['min(T,C)'])+ list(hl['min(T,C)'])+list(lh['min(T,C)'])+list(ll['min(T,C)'])          # used for caculating the general p_value
    list_group = ['hh']*len(hh['min(T,C)']) + ['hl']*len(hl['min(T,C)']) + ['lh']*len(lh['min(T,C)']) + ['ll']*len(ll['min(T,C)'])   #sed for caculating the general p_value
    list_vital_status_array = np.array(list_vital_status)
    list_min_T_C_array = np.array(list_min_T_C)
    list_group_array = np.array(list_group)
    single_p = pd.DataFrame()
    list_cluster = ['hh','hl','lh','ll']    #可改

    com5=combinations(list_cluster, 2)
    for key_a,key_b in com5:   
        pass



    list_a = [0,1,2,3,0,1,2,3]
    list_b = [0,0,0,0,1,1,1,1]
    list_a = [0,1,2,3]
    list_b = [0,0,0,0]

    if str(tk[1]) ==str(ct_list):

#                         hr()


        k_a_n = tk[2]
        k_a = 2
        k_b_n = tk[3]
        k_b = 1

        df_a = zip_cluster[k_a_n]
        df_a['group'] = k_a
        df_b = zip_cluster[k_b_n]
        df_b['group'] = k_b    
        df_ab = pd.concat([df_a,df_b],join='inner',axis=0)
        df_ab = df_ab[['min(T,C)','vital_status','group']]

        cph = CoxPHFitter()
        cph.fit(df_ab, duration_col='min(T,C)', event_col ='vital_status')
        HR = cph.hazard_ratios_[0]
        summary_  = cph.summary
        cox_p_lable = summary_['p'][0] 

        if HR<1:
            HR= 1/HR
        #logrank p
        T1 = list_min_T_C_array[list_group_array==tk[2]]
        T2 = list_min_T_C_array[list_group_array==tk[3]]
        E1 = list_vital_status_array[list_group_array==tk[2]]
        E2 = list_vital_status_array[list_group_array==tk[3]]
        if (T1.shape[0]!=0)&(T2.shape[0]!=0):

            results = logrank_test(T1, T2, event_observed_A=E1, event_observed_B=E2)
            s_logrank_p = results.p_value
           #
        sub_num += 1
        sub_num1 =sub_num-1
        clu_count = 0
        for clu_name,clu_data in zip_cluster.items():
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',clu_data['min(T,C)'].shape[0])

            if (clu_data['min(T,C)'].shape[0]!=0)&(clu_data['vital_status'].shape[0]!=0):

       
                legend_label=''
                if clu_name =='hh':
                    legend_label = 'H/H'
                if clu_name =='hl':
                    legend_label = 'H/L'
                if clu_name =='lh':
                    legend_label = 'L/H'
                if clu_name =='ll':
                    legend_label = 'L/L'
                plt.sca(ax[list_a[sub_num1]])    #ax[list_b[sub_num1],list_a[sub_num1]]
                kmf = KaplanMeierFitter()
                kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=f"{legend_label} ({len(clu_data['min(T,C)'])})")  #label=f"{legend_label} (n={len(clu_data['min(T,C)'])})"   
                median_ = np.round((kmf.median_survival_time_)/30,0  )   
                kmf = KaplanMeierFitter()
                kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=f"{legend_label} ({len(clu_data['min(T,C)'])}) MS:{median_}m")  #label=f"{legend_label} (n={len(clu_data['min(T,C)'])})"   
                kmf.plot(ci_show=False,show_censors=False,linewidth=0.5,ax=ax[list_a[sub_num1]])  #ax=ax[list_b[sub_num1],list_a[sub_num1]]
                ax[list_a[sub_num1]].legend(frameon=False,fontsize=6,handlelength=0.5,loc='upper right' )   # ax=ax[list_b[sub_num1],list_a[sub_num1]]
                ax[list_a[sub_num1]].set_xlabel('')   #ax=ax[list_b[sub_num1],list_a[sub_num1]]

                plt.title(f'{dataset_name}\n({media1_name}/{media2_name})',fontsize=8)


                statistic_result = multivariate_logrank_test(list_min_T_C, list_group, list_vital_status)     
                p_value_= np.round(statistic_result.p_value,10)                 #general p values

                count_index+=1

                clu_count+=1
                if clu_count==4:

                    plt.sca(ax[list_a[sub_num1]])     #  ax[list_b[sub_num1],list_a[sub_num1]]
                    plt.tick_params(pad=0.5,bottom=False, top=False, left=False, right=False,labelsize=8) #
                    plt.xlim(-200,7000)      # Change the limit of x-axis according to the desired plot.
                    plt.xticks([0,3000,6000])
                    x_left, x_right = plt.xlim()
                    print('----------------------------x_left, x_right---------------------------------------')
                    print(x_left, x_right)
                    y_bottom, y_top = plt.ylim()
                    x_coordinate = (x_right-x_left)*0.05+x_left
                    y_coordinate = (y_top-y_bottom)*0.2+y_bottom
                    y_coordinate_2 = (y_top-y_bottom)*0.14+y_bottom
                    y_coordinate_3 = (y_top-y_bottom)*0.08+y_bottom
                    y_coordinate_4 = (y_top-y_bottom)*0.02+y_bottom
                    axes = plt.gca()
                    axes.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
                    plt.text(
                            x_coordinate,
                            y_coordinate,
                            ("general p : "+f"{tk[2]}_{tk[3]}"),  #"general Log-rank "+"$\it{p}$ = "+f"{p_value_}"
#                                 transform=plt.gca().transAxes,
                            horizontalalignment="left",
                            fontsize=6
                    )#fontsize=4
                    plt.text(
                            x_coordinate,
                            y_coordinate_2,
                            ("logrank p : "+f"{s_logrank_p:.1E}"),  #"general Log-rank "+"$\it{p}$ = "+f"{p_value_}"
#                                 transform=plt.gca().transAxes,
                            horizontalalignment="left",
                            fontsize=6
                    )#fontsize=4
                    plt.text(
                            x_coordinate,
                            y_coordinate_3,
                            ("HR : "+f"{HR:.1E}"),  #"general Log-rank "+"$\it{p}$ = "+f"{p_value_}"
#                                 transform=plt.gca().transAxes,
                            horizontalalignment="left",
                            fontsize=6
                    )#fontsize=4
                    plt.text(
                            x_coordinate,
                            y_coordinate_4,
                            ("cox p : "+f"{cox_p_lable:.1E}"),  #"general Log-rank "+"$\it{p}$ = "+f"{p_value_}"
#                                 transform=plt.gca().transAxes,
                            horizontalalignment="left",
                            fontsize=6
                    )#fontsize=4


        if sub_num1==3:   


            fig.add_subplot(111,frameon=False)

            #plt.tick_params(pad=0.5,bottom=False, top=False, left=False, right=False) #labelsize=8
            plt.xlabel('Overall survival(Days)',fontsize=8)
            plt.ylabel('Survival probability',fontsize=8)
            plt.axis('off')
            p_f_path = os.path.join(save_path,'plot_survival_curve','paper_select')   
            make_path(p_f_path)

            p_survival_path = os.path.join(p_f_path,'survival_SIa.pdf')  ########
            
            plt.savefig(p_survival_path,bbox_inches = 'tight',dpi=300) 
            plt.show()
        


### Patient stratified by one cell type

#### Plot the cell type which log rank test is smaller than 0.05

In [ ]:
%%time



clinical_data_all = {}
logrank_p_df = pd.DataFrame()
median_s_t = pd.DataFrame()
hr_df = pd.DataFrame()
cox_p_df = pd.DataFrame()
info_df = pd.DataFrame()
count_except=0
count_ct =0
for dataset_name in beter_pre_list:

    save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\2023\survival_2group\1207\survival_2group_HR_deside_significantP'
    make_path(save_path)
    print(dataset_name,'********************************************')
    clinical_data_in_celltyppe ={}


    target_datasets=ori_datasets  #delect datasets.  ori_datasets: DeSide-predicted cell proportions;    my_datasets: The DeSide-predicted proportions of each cell type was normalized within the cohort.
    X = target_datasets[dataset_name]
    X = X.rename(columns={'Endothelial Cells':'Endo','Fibroblasts':'Fb','Macrophages':'mø','CD4 T':'C4','CD8 T':'C8','B Cells':'B'})
    cancer_meidan = X.median()
    cancer_meidan_path = os.path.join(save_path,'result_data','cellfrac',f'{dataset_name}')
    make_path(cancer_meidan_path)
    cancer_meidan_path = os.path.join(cancer_meidan_path,'ct_median.xlsx')
    cancer_meidan.to_excel(cancer_meidan_path)
    
    com2=combinations(X.columns, 2)


    for ct1 in X.columns:

        



        out_p_list =[]
#         out_p_list ={}
        sub_clinical_data={}
        
        med_ct1 = np.percentile(X[ct1],50)
        h_ =  X.loc[X[ct1]>med_ct1,:]
        l_ =  X.loc[X[ct1]<=med_ct1,:]
        print(ct1,'~~~~~~~~~~~~~~~~~~~~~~')
        print('Data number before goupe:',len(X))
        print('Data number before filtering:',len(h_)+len(l_))
        print('h_')
        h_ = delet_miss_in_clinical(h_)
        print('l_')
        l_ = delet_miss_in_clinical(l_)

        print('Data number after filtering:',len(h_)+len(l_))
        zip_cluster  = {'h_':h_,'l_':l_}


        T1 = h_['vital_status']
        T2 = l_['vital_status']
        E1 = ['h_']*len(h_['min(T,C)'])
        E2 = ['l_']*len(l_['min(T,C)'])
        print('11111111111111111111111111111')
        if (T1.shape[0]!=0)&(T2.shape[0]!=0):
            threshold_p = False
            results = logrank_test(T1, T2, event_observed_A=E1, event_observed_B=E2)
            if results.p_value<0.05:  #only select cell type for which the log_rank_test is smaller than 0.05
                count_ct+=1
                threshold_p = True
                logrank_p_df.loc[dataset_name,ct1]= results.p_value
                
                info_df.loc[count_ct,'cancer_type'] = dataset_name
                info_df.loc[count_ct,'cell type'] = ct1
                info_df.loc[count_ct,'logrank p value'] = results.p_value
                
            #calc hr
            d_a = h_.copy()
            d_b = l_.copy()
            d_a['group']=2  #The experimental group is labeled as "2" and serves as the numerator of the HR score.
            d_b['group']=1
            d_ab = pd.concat([d_a,d_b],axis=0,join='inner')
            d_ab = d_ab[['min(T,C)','vital_status','group']]
            try:
                cph = CoxPHFitter()
                cph.fit(d_ab, duration_col='min(T,C)', event_col ='vital_status')
            except:
                count_except+=1
                cph = CoxPHFitter(penalizer=0.1)
                cph.fit(d_ab, duration_col='min(T,C)', event_col ='vital_status')                
            hr_df.loc[dataset_name,ct1]= f"{cph.hazard_ratios_[0]:.3E}"
            summary_  = cph.summary
            cox_p_lable = summary_['p'][0]  
            
            if cph.hazard_ratios_[0]<1:   
                hr_larger_than_1 = 1/cph.hazard_ratios_[0]
                if threshold_p:
                    info_df.loc[count_ct,'HR'] = hr_larger_than_1
                    info_df.loc[count_ct,'cox_p'] = cox_p_lable
                    info_df.loc[count_ct,'better group'] = 'H'
            else:
                hr_larger_than_1 = cph.hazard_ratios_[0]
                if threshold_p:
                    info_df.loc[count_ct,'HR'] =cph.hazard_ratios_[0]
                    info_df.loc[count_ct,'cox_p'] = cox_p_lable
                    info_df.loc[count_ct,'better group'] = 'L'
            hr_df.loc[dataset_name,ct1+'larger_than1']= hr_larger_than_1
            cox_p_df.loc[dataset_name,ct1]= cox_p_lable
          

        
        plt.figure(figsize=(10,10))
        for clu_name,clu_data in zip_cluster.items():

            if (clu_data['min(T,C)'].shape[0]!=0)&(clu_data['vital_status'].shape[0]!=0):
                
                kmf = KaplanMeierFitter()
                kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=f"{clu_name}{ct1} (n={len(clu_data['min(T,C)'])})")  #str(len(sub_clinical['min(T,C)'])) #data_race['min(T,C)'][[0,1,2,4]]   
                
                median_ = kmf.median_survival_time_     
                median_s_t.loc[dataset_name,ct1+'_'+clu_name] = median_    #   
                kmf = KaplanMeierFitter()   #The reason for repeating this process is to add the "label" parameter in the kmf.fit() function.
                kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=f"{clu_name}{ct1} (n={len(clu_data['min(T,C)'])}) MS:{np.round(median_,0)}m")  #str(len(sub_clinical['min(T,C)'])) #data_race['min(T,C)'][[0,1,2,4]]   
                
                kmf.plot(ci_show=False)  #_false_interval


                plt.title(dataset_name)
                
                
            sub_clinical_data[clu_name] = clu_data
   
        if (T1.shape[0]!=0)&(T2.shape[0]!=0):
            p_value_= np.round(results.p_value,3)
        else:
            p_value_ = np.nan
        plt.text(
            0.1,
            0.2,
            ("Log-rank "+"$\it{p}$ = "+f"{p_value_}"),
            transform=plt.gca().transAxes,
            horizontalalignment="left",
            fontsize=12
        )
        plt.text(
        0.1,
        0.15,
        (f"HR :{hr_larger_than_1:.2e}"),
        transform=plt.gca().transAxes,
        horizontalalignment="left",
                    fontsize=12
    )   
        plt.text(
                0.1,
                0.1,
                (f"cox p :{cox_p_lable:.2e}"),
                transform=plt.gca().transAxes,
                horizontalalignment="left",
                            fontsize=12
            )
                
        clinical_data_in_celltyppe[str(ct1)] = sub_clinical_data
        

        if p_value_<0.05:   #only save the cell type which logrank p <0.05
            survival_plot_path = os.path.join(save_path,'plot_survival_curve',f'{dataset_name}')   
            make_path(survival_plot_path)
            survival_path = os.path.join(survival_plot_path,f'{str(ct1)}.jpg')
            plt.savefig(survival_path)  
    
    clinical_data_all[dataset_name]= clinical_data_in_celltyppe  
median_path = os.path.join(save_path,'plot_survival_curve','median_survival_time.xlsx')
    
median_s_t.to_excel(median_path)
hr_df_path =  os.path.join(save_path,'plot_survival_curve','hazard_ratios.xlsx')
hr_df.to_excel(hr_df_path)
p_p = os.path.join(save_path,'logrank_p.xlsx')
logrank_p_df.to_excel(p_p) 
c_p_p = os.path.join(save_path,'cox_p.xlsx')
cox_p_df.to_excel(c_p_p) 

info_df_path = os.path.join(save_path,'info_significant.xlsx')
info_df.to_excel(info_df_path) 

    
        

#### The cell type used in the paper.

In [ ]:


single_target_keys_use = [
    ('BLCA', 'CAFs','l'),
    ('KIRP','MyoFb','l'),
    ('HNSC','CD4 T','h'),
    ('LUAD','Mast Cells','h')
]

In [ ]:
%%time

#some values need to be note: sub_num，sub_num1，single_target_keys_use,  list_a,list_a，plt.subplots


 
sub_num = 0    #0   3   6  9  
# set_fig_style(font_family='Arial', font_size=8)
mpl.rcParams['font.family'] = 'Arial'
# mpl.rcParams['font.size'] = 6
# fig,ax = plt.subplots(2,5,figsize=(7,3),sharey= True,sharex= True)   #,sharey= True
# fig,ax = plt.subplots(2,3,figsize=(7,4.5),sharey= True,sharex= True)   #,sharey= True
# fig,ax = plt.subplots(4,3,figsize=(7,9),sharey= True,sharex= True)   #,sharey= True
fig,ax = plt.subplots(1,4,figsize=(7,1.7),sharey= True,sharex= True)   #,sharey= True

plt.subplots_adjust(hspace=0.25,wspace=0.05)

for tk in single_target_keys_use:
    
    dataset_name = tk[0]
    save_path = r'D:\EMT\EMT\jupyter_and_results_2022\cluster_and_survival_curve\results\all_results\2023\survival_2group\wenzhan'
    make_path(save_path)
    print(dataset_name,'********************************************')
    clinical_data_in_celltyppe ={}
    sing_p_in_celltyppe ={}
    out_p_in_celltyppe = {}
    overall_p_value ={}   
    target_datasets=ori_datasets  
    X = target_datasets[dataset_name]
    X = X.rename(columns={'Endothelial Cells':'Endo','Fibroblasts':'Fb','Macrophages':'mø','CD8 T':'C8','B Cells':'B',
                     'Non-plasma B cells':'Non-plasma B','Myofibroblasts':'MyoFb','Neutrophils':'Neut','Plasma B cells':'Plasma B'})   #改细胞类型的名字
#     X = X.rename(columns={'Endothelial Cells':'Endo','Fibroblasts':'Fb','Macrophages':'mø','CD4 T':'C4','CD8 T':'C8','B Cells':'B'})    #改细胞类型的名字

    
    com2=combinations(X.columns, 2)


    
    
    
   

    
    
    
    for ct1 in X.columns:


        med_ct1 = np.percentile(X[ct1],50)
        h_ =  X.loc[X[ct1]>med_ct1,:]
        l_ =  X.loc[X[ct1]<=med_ct1,:]
        print(ct1,'~~~~~~~~~~~~~~~~~~~~~~')
        print('Data number before goupe:',len(X))
        print('Data number before filtering:',len(h_)+len(l_))
        print('h_')
        h_ = delet_miss_in_clinical(h_)
        print('l_')
        l_ = delet_miss_in_clinical(l_)

        print('Data number after filtering:',len(h_)+len(l_))
        zip_cluster  = {'h_':h_,'l_':l_}


        T1 = h_['vital_status']
        T2 = l_['vital_status']
        E1 = ['h_']*len(h_['min(T,C)'])
        E2 = ['l_']*len(l_['min(T,C)'])

        
        
        single_p = pd.DataFrame()
        if (T1.shape[0]!=0)&(T2.shape[0]!=0):
            results = logrank_test(T1, T2, event_observed_A=E1, event_observed_B=E2)
            single_p.loc[str('h_'),str('l_')] = results.p_value

        
        
        

   
        list_a = [0,1,2,3]   #
        list_b = [0,0,0,0]

        if tk[1]==ct1:
            #算 HR
            d_a = h_.copy()
            d_b = l_.copy()
            if tk[2]=='h':
                d_a['group']=2
                d_b['group']=1
            else:
                d_a['group']=1
                d_b['group']=2                   
            d_ab = pd.concat([d_a,d_b],axis=0,join='inner')
            d_ab = d_ab[['min(T,C)','vital_status','group']]

            cph = CoxPHFitter()
            cph.fit(d_ab, duration_col='min(T,C)', event_col ='vital_status')
            HR = cph.hazard_ratios_[0]
            if HR<1:
                HR = 1/HR
            summary_  = cph.summary
            cox_p_lable = summary_['p'][0]





            sub_num += 1
            sub_num1 =sub_num-1
            clu_count = 0

            for clu_name,clu_data in zip_cluster.items():

                if (clu_data['min(T,C)'].shape[0]!=0)&(clu_data['vital_status'].shape[0]!=0):
                    legend_label=''
                    if clu_name =='h_':
                        legend_label = f"H (n={len(clu_data['min(T,C)'])})"
                    if clu_name =='l_':
                        legend_label = f"L (n={len(clu_data['min(T,C)'])})"


                    plt.sca(ax[list_a[sub_num1]])
#                     plt.sca(ax[list_b[sub_num1],list_a[sub_num1]])
                    ############################
                    kmf = KaplanMeierFitter()
                    kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=legend_label)  #label=f"{clu_name}{ct1} (n={len(clu_data['min(T,C)'])})"
                    median_ = (kmf.median_survival_time_)/30     #Unit: months 
                    kmf = KaplanMeierFitter()      #
                    kmf.fit(clu_data['min(T,C)'], clu_data['vital_status'], label=f'{legend_label} MS:{np.round(median_,0)}m')  #label=f"{clu_name}{ct1} (n={len(clu_data['min(T,C)'])})"

                    kmf.plot(ci_show=False,show_censors=False,linewidth=0.5,ax=ax[list_a[sub_num1]])  #ax=ax[list_b[sub_num1],list_a[sub_num1]]
                    plt.title(f'{dataset_name} ({ct1})',fontsize=8)
                    ax[list_a[sub_num1]].legend(fontsize=6 )   #fontsize=5  ax=ax[list_b[sub_num1],list_a[sub_num1]]
                    ax[list_a[sub_num1]].set_xlabel('')  #ax=ax[list_b[sub_num1],list_a[sub_num1]]


                    clu_count+=1
                    ##########################0000
                    if clu_count==2:
                        plt.sca(ax[list_a[sub_num1]])   #ax[list_b[sub_num1],list_a[sub_num1]]
                        plt.legend(frameon=False,fontsize=6,handlelength=0.7,loc='upper right' )
                        plt.tick_params(pad=0.5,bottom=False, top=False, left=False, right=False,labelsize=8)   #To adjust the distance between the axis labels and the axis, hide the tick lines, and change the font size of the labels.
#                         plt.xlabel(fontsize=8)
#                         plt.ylabel(fontsize=8)
                        ax1=plt.gca()   
                        ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))   # To display the decimal places on the y-axis as one decimal point

                        plt.xlim(-200,9000)     #To ensure that all the x-axis have the same limit, adjust the plot settings to set a consistent range for the x-axis across all plots.
                        plt.xticks([0,4000,8000])
                        x_left, x_right = plt.xlim()
                        print(tk[0])
                        print('plt.xlim :',x_left, x_right)
                        y_bottom, y_top = plt.ylim()
                        x_coordinate = (x_right-x_left)*0.05+x_left
                        y_coordinate = (y_top-y_bottom)*0.17+y_bottom
                        y_coordinate_2 = (y_top-y_bottom)*0.09+y_bottom
                        y_coordinate_3 = (y_top-y_bottom)*0.02+y_bottom
                        if (T1.shape[0]!=0)&(T2.shape[0]!=0):
                            p_value_= results.p_value
                        else:
                            p_value_ = np.nan
                        plt.text(
                                x_coordinate,
                                y_coordinate,
                                ("logrank p : "+f"{p_value_:.1E}"),   #("Log-rank "+"$\it{p}$ = "+f"{p_value_}"),   
            #                                 transform=plt.gca().transAxes,
                                horizontalalignment="left",
                                fontsize=6
                        )#fontsize=2
                        plt.text(
                                x_coordinate,
                                y_coordinate_2,
                                ("HR : "+f"{HR:.1E}"),   #("Log-rank "+"$\it{p}$ = "+f"{p_value_}"),   
            #                                 transform=plt.gca().transAxes,
                                horizontalalignment="left",
                                fontsize=6)
                        plt.text(
                                x_coordinate,
                                y_coordinate_3,
                                ("cox p : "+f"{cox_p_lable:.1E}"),   #("Log-rank "+"$\it{p}$ = "+f"{p_value_}"),   
            #                                 transform=plt.gca().transAxes,
                                horizontalalignment="left",
                                fontsize=6)
            #                     sub_clinical_data[clu_name] = clu_data
            ##################################
            if  sub_num1==3:     #2  5   8  11

                fig.add_subplot(111,frameon=False)      #?
                plt.tick_params(labelcolor='none',which = 'both',top=False,bottom=False,left=False,right=False,labelsize=8)
                plt.xlabel('Overall survival(Days)',fontsize=8)
                plt.ylabel('Survival probability',fontsize=8)
                p_f_path = os.path.join(save_path,'paper_select')   
                make_path(p_f_path)
                p_survival_path = os.path.join(p_f_path,'one_ct_1.pdf')   #################
                plt.savefig(p_survival_path,bbox_inches = 'tight') 
                    
                            
                        
                        
                        
                        
    
        

In [ ]:
# def judge_max_suvival(ma,subcluster):
#     judge_ = []   # "True" indicates that the patients' prognosis for this curve is better. 
# #     ma = 'hh'
# #     subcluster = survival_function_in_dataset['READ']["('Fibroblasts', 'Macrophages')"]
#     all_cluster = list(subcluster.keys())
#     all_cluster.remove(ma)  # 要比较的cluster
#     cc =  subcluster[ma]

#     cc.index.max()
    
#     for i in all_cluster:
#         another =  subcluster[i]
#         if another.index.max()>=cc.index.max():
#             another_survival_rate = another.loc[another[another.index>=cc.index.max()].index.min(),another.columns[0]]
#             cc_survival_rate = cc.loc[cc.index.max(),cc.columns[0]]   #
#     #         print(i)
#     #         print("another.index.max()>=cc.index.max():")


#             if another_survival_rate<=cc_survival_rate:
#                 judge=True
#                 print(another_survival_rate,'<=',cc_survival_rate)
#             else :
#                 judge=False
#         else:
#     #         print(i)
#     #         print("another.index.max()<cc.index.max():")

#             another_survival_rate = another.loc[another.index.max(),another.columns[0]]
#             cc_survival_rate = cc.loc[cc[cc.index>=another.index.max()].index.min(),cc.columns[0]]   #
#             if another_survival_rate<=cc_survival_rate:
#                 judge=True
#     #             print(another_survival_rate,'<=',cc_survival_rate)
#             else:
#                 judge=False

#         judge_.append(judge)
#     judge_ = np.array(judge_)
#     judge_a = judge_.all()
#     return judge_a


# def judge_min_suvival(ma,subcluster):
#     judge_ = []   # True 表示这条生存曲线的确是最小的
# #     ma = 'hh'
# #     subcluster = survival_function_in_dataset['READ']["('Fibroblasts', 'Macrophages')"]
#     all_cluster = list(subcluster.keys())
#     all_cluster.remove(ma)  # 要比较的cluster
#     cc =  subcluster[ma]

#     cc.index.max()
#     for i in all_cluster:
        
#         another = subcluster[i]
#         if another.index.max()>=cc.index.max():
# #             another_survival_rate = another.loc[another[another.index<=cc.index.max()].index.max(),another.columns[0]]    #
#             another_survival_rate = another.loc[another[another.index<=cc.index.max()].index.min(),another.columns[0]]  #
#             cc_survival_rate = cc.loc[cc.index.max(),cc.columns[0]]    #
#             if another_survival_rate>=cc_survival_rate:
#                 judge=True
#             else :
#                 judge=False
#         else:
#             another_survival_rate = another.loc[another.index.max(),another.columns[0]]
#             cc_survival_rate = cc.loc[cc[cc.index>=another.index.max()].index.min(),cc.columns[0]]   #
#             if another_survival_rate>=cc_survival_rate:
#                 judge=True
#             else:
#                 judge=False
#         judge_.append(judge)
#     judge_ = np.array(judge_)
#     judge_a = judge_.all()
#     return judge_a

# def judge_two_survial(min_,max_,subcluster):
#     '''如果这里的最大最小的生存率都与50%生存率预测的一致，则为True
#     '''
#     judge_mi = judge_min_suvival(ma=min_,subcluster=subcluster)
#     judge_ma = judge_max_suvival(ma=max_,subcluster=subcluster)
#     list_ = [judge_mi,judge_ma]
#     list_ = np.array(list_)
#     judge_a = list_.all()
#     return judge_a


